# Pushkin as a Traveller

Pushkin grew up in Moscow and St. Petersburg. In 1819 he was exiled, at the age of nineteen, to a small town called Kishinev and then he was moved to his family estate in Ekaterinoslav. He remained in exile for nearly seven years until 1826 when Tsar Nicholas came to power. Below you can see a map of his travels throughout Russia.

<img src="https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/pushkin/O.Kiprensky_-_A.S.Pushkin.jpg" alt="Portrait of A.S. Pushkin" style="width: 300px;"/>

[Original version](https://www.rgo.ru/ru/article/pushkin-puteshestvennik-linii-zhizni-linii-na-karte) of the graph that you can see below.

Also, see the [short journey through his biography](https://pushkinland.ru/2018/english/push1.php).

Some data was collected from other sources, such as:

- [https://en.wikipedia.org/wiki/Alexander_Pushkin](https://en.wikipedia.org/wiki/Alexander_Pushkin)
- [http://hronika.su/pushkin-a-s](http://hronika.su/pushkin-a-s)

In [1]:
import pandas as pd

from lets_plot import *

In [2]:
LetsPlot.setup_html()

In [3]:
def append_stay_time(places_df, moves_df):
    def city_name_to_years(name):
        moves_df.sort_values(by="year", inplace=True)
        departure_years = moves_df[moves_df["departure"] == name]["year"].to_list()
        arrival_years = moves_df[moves_df["arrival"] == name]["year"].to_list()
        if len(departure_years) > 0 and min(departure_years) < min(arrival_years):
            arrival_years = [min(departure_years)] + arrival_years
        def arrival_year_to_years_range(arrival_year):
            departure_year = next((year for year in departure_years if year >= arrival_year), None) \
                             if len(departure_years) > 0 else arrival_year
            if departure_year is None:
                departure_year = arrival_year
            return arrival_year, departure_year
        all_years_range = sorted(
            [arrival_year_to_years_range(arrival_year) for arrival_year in arrival_years],
            key=lambda pair: pair[1] - pair[0],
            reverse=True
        )
        year_ranges = []
        for year_range in all_years_range:
            outer_range = next((
                big_year_range
                for big_year_range in year_ranges
                if big_year_range[0] <= year_range[0] and year_range[1] <= big_year_range[1]
            ), None)
            if outer_range is None:
                year_ranges.append(year_range)
        return ", ".join([
            str(year_range[0]) if year_range[0] == year_range[1] else "{0}-{1}".format(*year_range)
            for year_range in sorted(year_ranges, key=lambda yrange: yrange[0])
        ])

    return places_df.assign(years=[city_name_to_years(name) for name in places_df["name"]])

In [4]:
city_size = 3
big_city_size = 6
big_cities = ["Moscow", "Petersburg"]
size_replace = {True: big_city_size, False: city_size}

raw_places_df = pd.read_csv("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/pushkin/places.csv")

raw_moves_df = pd.read_csv("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/pushkin/moves.csv")

moves_df = raw_moves_df.merge(raw_places_df, left_on="departure", right_on="name")\
                       .rename(columns=dict(longitude="from_lon", latitude="from_lat"))\
                       .drop(columns=["name"])\
                       .merge(raw_places_df, left_on="arrival", right_on="name")\
                       .rename(columns=dict(longitude="to_lon", latitude="to_lat"))\
                       .drop(columns=["name"])
moves_df = moves_df.assign(
    size_start=moves_df["departure"].isin(big_cities).replace(size_replace),
    size_end=moves_df["arrival"].isin(big_cities).replace(size_replace),
)

places_df = append_stay_time(raw_places_df, raw_moves_df)
places_df = places_df.assign(size=places_df["name"].isin(big_cities).replace(size_replace))

/var/folders/rh/rpmcnm9j75x10cnx_sfmclsr0000gn/T/ipykernel_39801/2394508422.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  size_start=moves_df["departure"].isin(big_cities).replace(size_replace),
/var/folders/rh/rpmcnm9j75x10cnx_sfmclsr0000gn/T/ipykernel_39801/2394508422.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  size_end=moves_df["arrival"].isin(big_cities).replace(size_replace),
/var/folders/rh/rpmcnm9j75x10cnx_sfmclsr0000gn/T/ipykernel_39801/2394508422.py:22: FutureWarning: Downcasting behavior in `replace` is d

In [5]:
ggplot() + \
    geom_livemap(const_size_zoomin=3) + \
    geom_curve(aes(x="from_lon", y="from_lat", xend="to_lon", yend="to_lat", color="path", \
                   size_start="size_start", size_end="size_end"),
               data=moves_df, curvature=-.15, size=.75,
               tooltips=layer_tooltips().title("Trip name").line("@path"),
               arrow=arrow(type='closed', length=10, angle=14)) + \
    geom_point(aes(x="longitude", y="latitude", size="size"), data=places_df, shape=21, fill="white",
               tooltips=layer_tooltips().title("@name").line("Visited in @years")) + \
    scale_color_manual(name="Trip name", values=['#addd8e', '#e34a33', '#8856a7', '#2c7fb8',
                                                 '#1c9099', '#006d2c', '#fec44f', '#636363']) + \
    scale_size_identity(guide='none') + \
    ggtitle("Alexander Pushkin's Trips") + \
    ggsize(900, 900) + \
    theme(axis_title='blank', axis_text='blank', axis_ticks='blank', axis_line='blank',
          plot_title=element_text(size=20, margin=[20, 0, 0, 0]),
          legend_position=[1, 1], legend_justification=[1, 1])